In [ ]:
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
%matplotlib inline
import keras 
from keras.api.models import Sequential
from keras.api.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report
import pandas as pd
np.random.seed(69) #for funzies


#Data

In [ ]:
from keras.api.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

# Visualizando Exemplos

In [ ]:
num_classes = 10
f, ax = plt.subplots(1, num_classes, figsize=(20,20))
for i in range(0, num_classes):
    sample = x_train[y_train==i][0]
    ax[i].imshow(sample, cmap='gray')
    ax[i].set_title("label: {}".format(i), fontsize= 16)


In [ ]:
#discretizando em classes numericas para que o modelo nao de numeros quebrados
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test= keras.utils.to_categorical(y_test, num_classes)

In [ ]:
for i in range(10):
    print(y_train[i])

# Preparando Dados

In [ ]:
# Normalizacao
x_train = x_train/255
x_test = x_test/255

In [ ]:
#transformando dados em um vetor ao invez de matriz
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)
print(x_train.shape)

# Criando a RNA 

In [ ]:
model = Sequential()
model.add(Dense(units=128, input_shape=(784,), activation='relu'))#camada de entrada
model.add(Dense(units=128, activation='relu')) # Camada Intermediaria
model.add(Dense(units=128, activation='relu')) # Camada Intermediaria
model.add(Dense(units=128, activation='relu')) # Camada Intermediaria

model.add(Dropout(0.45)) #Testar depois com diferentes intensidades
model.add(Dense(units=10, activation='softmax')) # Camada de saida

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Treinando a Rede Neural 

In [ ]:
batch_size = 128
epochs = 120
model.fit(x=x_train,y=y_train, batch_size=batch_size, epochs=epochs)

# Avaliacao

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Loss: {}, Test accuracy: {}".format(test_loss, test_acc))

In [ ]:
# Adiciona ruído gaussiano
noise_factor = 0.4
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [ ]:
test_loss, test_acc = model.evaluate(x_test_noisy, y_test)
print("Test Loss: {}, Test accuracy: {}".format(test_loss, test_acc))

In [ ]:
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
#print(y_pred)
#print(y_pred_classes)

In [ ]:
y_pred_noisy_probs = model.predict(x_test_noisy)
y_pred_classes_noisy = np.argmax(y_pred_noisy_probs, axis=1)
#print(y_pred)
#print(y_pred_noisy_probs)

In [ ]:
random_x =  np.random.choice(len(x_test))
x_sample = x_test[random_x]
y_true = np.argmax(y_test, axis=1)
y_sample_true = y_true[random_x]
y_sample_pred_class = y_pred_classes[random_x]

plt.title("predicted: {}, true: {}".format(y_sample_pred_class, y_sample_true, fontsize=16))
plt.imshow(x_sample.reshape(28,28), cmap = 'gray')

In [ ]:



# Seleciona N exemplos aleatorios
num_examples = 15
#indices = np.arange(num_examples)
indices = np.random.choice(10000, size=num_examples, replace=False)
# Previsões para essas imagens
preds_clean = np.argmax(model.predict(x_test[indices]), axis=1)
preds_noisy = np.argmax(model.predict(x_test_noisy[indices]), axis=1)
true_labels = np.argmax(y_test[indices], axis=1)

# Cria a figura
plt.figure(figsize=(20, 4))

for i, idx in enumerate(indices):
    # Original
    plt.subplot(2, num_examples, i + 1)
    plt.imshow(x_test[idx].reshape(28, 28), cmap='gray')
    plt.title(f"Real: {true_labels[i]}\nPred: {preds_clean[i]}")
    plt.axis('off')
    
    # Com ruído
    plt.subplot(2, num_examples, i + 1 + num_examples)
    plt.imshow(x_test_noisy[idx].reshape(28, 28), cmap='gray')
    plt.title(f"Noisy Pred: {preds_noisy[i]}")
    plt.axis('off')

plt.suptitle("Linha 1: Imagens originais  |  Linha 2: Imagens com ruído", fontsize=16)
plt.tight_layout()
plt.show()


Metricas para avaliar, 
Acuracia, 
Perdas ,
Especificidade,
Classification Report SKlearn,


# Matriz de Confusao

In [ ]:
confusion_matrix = sk.metrics.confusion_matrix(y_true, y_pred_classes)

fig , ax = plt.subplots(figsize=(30,15))
ax = sns.heatmap(confusion_matrix, annot=True, fmt='d', ax=ax, cmap="Blues")
ax.set_xlabel('predict')
ax.set_ylabel('True')
ax.set_title('Matriz De confusao');

In [ ]:
conf_matrix_noisy = sk.metrics.confusion_matrix(y_true, y_pred_classes_noisy)
fig , ax = plt.subplots(figsize=(30,15))
ax = sns.heatmap(conf_matrix_noisy, annot=True, fmt='d', ax=ax, cmap="Greens")
ax.set_xlabel('predict')
ax.set_ylabel('True')
ax.set_title('Matriz De confusao Com Ruido');

In [ ]:


# Previsões sem ruído
y_pred_probs = model.predict(x_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Relatório como DataFrame
report_dict = classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report_dict).transpose()
df_report = df_report.drop(['accuracy'], errors='ignore')  # opcional

# Heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(df_report.iloc[:, :-1], annot=True, fmt=".2f", cmap="Blues")
plt.title("Classification Report - Sem Ruído")
plt.show()



In [ ]:

# Previsões com ruído
y_pred_noisy_probs = model.predict(x_test_noisy)
y_pred_noisy = np.argmax(y_pred_noisy_probs, axis=1)

# Relatório como DataFrame
report_dict_noisy = classification_report(y_true, y_pred_noisy, output_dict=True)
df_report_noisy = pd.DataFrame(report_dict_noisy).transpose()
df_report_noisy = df_report_noisy.drop(['accuracy'], errors='ignore')

# Heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(df_report_noisy.iloc[:, :-1], annot=True, fmt=".2f", cmap="Greens")
plt.title("Classification Report - Com Ruído")
plt.show()


# Investigando os Erros (curiosidade)

In [ ]:
errors = (y_pred_classes - y_true != 0)
y_pred_classes_errors = y_pred_classes[errors]
y_pred_errors = y_pred[errors]
y_pred_noisy_errors = y_pred_noisy[errors]
x_test_noisy_errors = x_test_noisy[errors]
y_true_errors = y_true[errors]
x_test_errors = x_test[errors]

In [ ]:
y_pred_errors_probability = y_pred_errors  # Since y_pred_errors is 1D, this is already the max probability
true_probability_errors = y_true_errors  # Directly use y_true_errors as the true probabilities
diff_errors_pred_true = y_pred_errors_probability - true_probability_errors
diff_errors_pred_Noisy_true = y_pred_noisy_errors - true_probability_errors

# Get list of indices of sorted differences
sorted_idx_diff_errors = np.argsort(diff_errors_pred_true)
top_idx_diff_errors = sorted_idx_diff_errors[-15:]  # 15 last ones

sorted_idx_diff_errors_noisy = np.argsort(diff_errors_pred_Noisy_true)
top_idx_diff_errors_noisy = sorted_idx_diff_errors_noisy[-15:]  # 15 last ones




In [ ]:

# Show Top Errors
num = len(top_idx_diff_errors)
f, ax = plt.subplots(1, num, figsize=(60,60))

for i in range(0, num):
  idx = top_idx_diff_errors[i]
  sample = x_test_errors[idx].reshape(28,28)
  y_t = y_true_errors[idx]
  y_p = y_pred_classes_errors[idx]
  ax[i].imshow(sample, cmap='gray')
  ax[i].set_title("Predicted label :{}\nTrue label: {}".format(y_p, y_t), fontsize=22)

In [ ]:
# Show Top Errors
num = len(top_idx_diff_errors_noisy)
f, ax = plt.subplots(1, num, figsize=(60,60))

for i in range(0, num):
  idx = top_idx_diff_errors_noisy[i]
  sample = x_test_noisy_errors[idx].reshape(28,28)
  y_t = y_true_errors[idx]
  y_p = y_pred_noisy_errors[idx]
  ax[i].imshow(sample, cmap='gray')
  ax[i].set_title("Predicted label :{}\nTrue label: {}".format(y_p, y_t), fontsize=22)